In [97]:
### IMPORTS AND DRIVER SETUP

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, UnexpectedAlertPresentException, NoAlertPresentException
from selenium.common.exceptions import SessionNotCreatedException, WebDriverException
from selenium.webdriver.chrome.options import Options

import time
import pandas as pd
import os
import re
import sys
import linecache
import numpy as np
import datefinder

from bs4 import BeautifulSoup as soup
from pandas import NA

chrome_driver_binary = "/Users/andrin/Library/CloudStorage/OneDrive-UniversitätZürichUZH/Research Assistance/Web Scraping/chromedriver"

driver = webdriver.Chrome(chrome_driver_binary)



In [98]:
### SCRAPING ARTICLES FUNCTION

def Scrape(article_urls,scraped_articles=None):
    
    if scraped_articles is None:
        scraped_articles = []


    error_articles = []

    for article_number, article in enumerate(article_urls):

        driver.get(article)

        time.sleep(3)

        #Grab All Text Content
        all_text = str(driver.find_element_by_xpath("/html/body").text)

        #Find Headline
        try:
            headline = str(driver.find_element_by_xpath("//*[@class='enHeadline']").text).replace("FT.com site : ","")
        except NoSuchElementException:
            headline = pd.NA
            print(article_number+1," NO HEADLINE DETECTED - CONTINUING...")
            

        #Find Author
        try:
            author = driver.find_element_by_xpath("//*[@class='author']").text
        except NoSuchElementException:
            author = pd.NA
            print(article_number+1," NO AUTHOR DETECTED - CONTINUING...")
        
        #Find Text Paragraphs
        try:
            text_lines = driver.find_elements_by_xpath("//*[@class='articleParagraph enarticleParagraph']")
        except NoSuchElementException:
            text_lines = pd.NA
            print(article_number+1," !!! WARNING: NO TEXT CONTENT DETECTED - CONTINUING...")

        
        #Find Date
        dates_in_text = list(datefinder.find_dates(all_text, strict = True))
        if len(dates_in_text) > 0:
            date = str(dates_in_text[0])[:10]
        else:
            date = pd.NA

        #Aggregate Article Text Contents 
        content = ""
        for line in text_lines:
            content = content + str(line.text)+"\n"

        #Article Did Not Load Warning
        if "We are unable to process your request at this time. Please try again in a few minutes." in str(all_text):
            print("ERROR: ARTICLE DID NOT LOAD !!!")
            time.sleep(3)
            driver.refresh()
            time.sleep(3)
            error_articles.append(article)
            error_message = "ARTICLE DID NOT LOAD"
        else:
            error_message = pd.NA

        #Create Article Element and Add to List of Scraped Articles
        article_element = [article_number+1,headline,author,date,content,error_message]
        #print(article_element)
        
        scraped_articles.append(article_element)


        time.sleep(1)

        print("Article Number: ", article_number+1, " Progress: ", round((article_number+1)*100/len(article_urls),2), "%")

    if len(error_articles) > 0:
        Scrape(error_articles,scraped_articles)

    print(len(scraped_articles))
            
    return scraped_articles


In [100]:
url = 'https://ezproxy.uzh.ch/login?url=https://global.factiva.com/en/sess/login.asp?XSID=S004cVk3HVpZXJyMTZyMTMtM96mMTImMtmm5Ff9R9apRsJpWVFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFB'

driver.get(url)

In [102]:
url = 'https://ezproxy.uzh.ch/login?url=https://global.factiva.com/en/sess/login.asp?XSID=S004cVk3HVpZXJyMTZyMTMtM96mMTImMtmm5Ff9R9apRsJpWVFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFB'

all_articles_aggregate_df = pd.DataFrame(columns=['ARTICLE_NUMBER','TITLE','AUTHOR', 'DATE','TEXT','ERRORS','BANK'])

banks_to_search = [
#"BB&T Corp",
#"Bank of America",
#"Bank of New York Mellon CIT Group",
#"Citi",
#"Citizens Financial Group",
#"Comerica Inc",
"Fifth Third Bank",
"First Bank",
"General Electric Capital",
#"Goldman Sachs",
"Hibernia Corp",
"Huntington Bancshares",
"JP Morgan",
"Jefferies",
"Key Bank",
"M&T Bank",
"Merrill Lynch International",
"Morgan Stanley",
"Morgan Stanley MUFG Loan Partners LLC Northern Trust Corp",
"PNC Bank",
"Regions Bank",
"SVB Financial Group",
"SouthTrust Bank",
"State Street Bank",
"SunTrust Bank",
"US Bancorp",
"Wells Fargo",
"Zions Bancorporation"]

for bankname in banks_to_search:
    print("BANK TO SEARCH FOR: ", bankname)

    driver.get(url)

    search = True

    time.sleep(10)

    focused_elem = driver.switch_to.active_element
    search_query = bankname + " and " + "climate change"
    focused_elem.send_keys(search_query)

    time.sleep(2)

    select_dupli = Select(driver.find_element_by_id('isrd'))
    select_dupli.select_by_visible_text('Identical')

    select_daterange = Select(driver.find_element_by_id('dr'))
    select_daterange.select_by_visible_text('All Dates')

    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//button[text()="All Publications"]'))).click()
    #select_daterange.select_by_visible_text('Remove')

    focused_elem.send_keys(Keys.ENTER)
    try:
        #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//li[@title='Financial Times (Available through Third Party Subscription Services) - All sources']"))).click()
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//li[@title='The Wall Street Journal - All sources']"))).click()
    except TimeoutException as te:
        time.sleep(2)
        print("NO ARTICLES FROM THIS SOURCE")
        search = False

    time.sleep(5)

    if search == True:

        inspect_urls = []

        while True:
            try:
                elems = driver.find_elements_by_xpath("//a[@href]")
                for elem in elems:
                    inspect_urls.append(elem.get_attribute("href"))

                time.sleep(5)

                driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.nextItem"))))
                driver.find_element(By.CSS_SELECTOR, 'a.nextItem').click()
                print("Navigating to Next Page")

                time.sleep(5)

            except UnexpectedAlertPresentException as uap:
                try:
                    driver.switch_to.alert.accept()
                    print(uap)
                    driver.refresh()
                except NoAlertPresentException as nap:
                    print(nap)
                    driver.refresh()

                
            except (TimeoutException, WebDriverException) as e:
                time.sleep(2)
                print("Last page reached")
                break

        all_article_links = []

        for link in inspect_urls:
            if "accessionno" in link:
                all_article_links.append(link)

        all_articles = np.array(all_article_links)


        short_links = []

        for link in all_article_links:
            short_link = link[:79]
            short_links.append(short_link)


        unique_indices = sorted(np.unique(short_links, return_index=True)[1].tolist())

        unique_articles = all_articles[unique_indices].tolist()

        #### SCRAPE TEXT

        #test_articles_to_find = unique_articles[:3]

        article_list_scraped = Scrape(unique_articles)
        articles_df = pd.DataFrame(article_list_scraped)


        articles_df.columns = ['ARTICLE_NUMBER','TITLE','AUTHOR', 'DATE','TEXT','ERRORS']

        articles_df['BANK'] =  bankname

        all_articles_aggregate_df = pd.concat([all_articles_aggregate_df,articles_df],ignore_index=True)
    else:
        pass

all_articles_aggregate_df = all_articles_aggregate_df.drop(all_articles_aggregate_df.columns[[5]] , axis=1)
all_articles_aggregate_df.to_csv("US_BANKS_TESTS.csv", encoding='utf-8', index=False)

BANK TO SEARCH FOR:  Fifth Third Bank


TimeoutException: Message: 


In [80]:
all_articles_aggregate_df = all_articles_aggregate_df.drop(all_articles_aggregate_df.columns[[5]] , axis=1)
all_articles_aggregate_df.to_csv("US_BANKS_TESTS2.csv", encoding='utf-8', index=False)

In [95]:
df_p1 = pd.read_csv("US_BANKS_P1.csv")
df_p2 = pd.read_csv("US_BANKS.csv")

DF_US_BANKS = pd.concat([df_p1,df_p2],ignore_index=True)
DF_US_BANKS.to_csv("US_BANKS_ALL.csv", encoding='utf-8', index=False)

In [ ]:
print(all_articles[1][:166])

short_links = []

for link in all_article_links:
    short_link = link[:166]
    short_links.append(short_link)


unique_indices = sorted(np.unique(short_links, return_index=True)[1].tolist())

unique_articles = all_articles[unique_indices].tolist()

print(len(unique_articles))
print(unique_articles)

In [30]:
### AGGREGATING THE DATA FOR ALL BANKS EVER SEARCHED FOR

art_df_firstpart = pd.read_csv("BANKS_FACTIVA.csv")
art_df_firstpart = art_df_firstpart.drop(art_df_firstpart.columns[[0]] , axis=1)

art_df_secondpart = pd.read_csv("all_articles_aggregate_df1.csv")

all_banks = pd.concat([art_df_firstpart,art_df_secondpart,all_articles_aggregate_df],ignore_index=True)


In [32]:
all_banks.to_csv("ALL_BANKS.csv", encoding='utf-8', index=False)

In [ ]:
### ALL BANKS TO ADDITIONALLY SEARCH FOR

banks_to_search = ["Royal Bank of Scotland",
"Bank of Scotland",
"National Westminster",
"Lloyds Bank",
"CIT Bank",
"Midland Bank",
"European Bank for Reconstruction & Development",
"Allied Irish Bank",
"Rabobank",
"Standard Bank",
"Scotiabank Europe",
"Barclays de Zoete Wedd",
"Nomura Bank",
"Abbey National Treasury Services",
"Investec Bank",
"Santander UK",
"Greenwich NatWest"]

In [ ]:
banks_to_search = [
"CIT Bank",
"Midland Bank",
"European Bank for Reconstruction & Development",
"Allied Irish Bank",
"Rabobank",
"Standard Bank",
"Scotiabank Europe",
"Barclays de Zoete Wedd",
"Nomura Bank",
"Abbey National Treasury Services",
"Investec Bank",
"Santander UK",
"Greenwich NatWest"]

In [ ]:
banks_to_search = ["BB&T Corp",
"Bank of America",
"Bank of New York Mellon CIT Group",
"Citi",
"Citizens Financial Group",
"Comerica Inc",
"Fifth Third Bank",
"First Bank",
"General Electric Capital",
"Goldman Sachs",
"Hibernia Corp",
"Huntington Bancshares",
"JP Morgan",
"Jefferies",
"Key Bank",
"M&T Bank",
"Merrill Lynch International",
"Morgan Stanley",
"Morgan Stanley MUFG Loan Partners LLC Northern Trust Corp",
"PNC Bank",
"Regions Bank",
"SVB Financial Group",
"SouthTrust Bank",
"State Street Bank",
"SunTrust Bank",
"US Bancorp",
"Wells Fargo",
"Zions Bancorporation"]